In [144]:
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance
from qdrant_client.http.models import PointStruct
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain_openai import ChatOpenAI
from typing import List
import os

load_dotenv()
collection_name = "prawo_wodne"

ChatOpenAI.available_models

AttributeError: available_models

In [ ]:
def multi_query(user_query: str, number_of_queries: int) -> str:
    """Using user query creates a multiple versions of query for better document gather"""
    MULTI_QUERY_PROMPT = (
        "Jesteś asystentem AI. Twoim zadaniem jest wygenerowanie {number_of_queries} różnych wersji podanego pytania w celu późniejszego pobrania danych z bazy danych."
        "Poprzez wygenerowanie {number_of_queries} różnych wersji jego pytania, masz pomóc użytkownik na przezwyciężenie różnych trudności w wyszukaniu danych jakie mogą wynikać z jego pytania."
        "Podaj tylko i wyłącznie nowe pytania odseparowanie poprzez nowe linie, podaj to w taki sposób"
        "Oto {number_of_queries} nowe pytania"
        "Pytanie1: treść pytania"
        "Pytanie2: treść pytania"
        "itp."
        "Oryginalne pytanie {user_query}"
    )
    prompt = MULTI_QUERY_PROMPT.format(
        number_of_queries=number_of_queries, user_query=user_query
    )
    model_response = model.invoke([{"role": "assistant", "content": prompt}]).content
    return model_response.split("\n\n")[-number_of_queries:]

In [132]:
import os
from typing import List

from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, PointStruct, VectorParams

collection_name = "prawo_wodne"

"""
This file is responsible for loading a pdf file from hugginface, transforming it to embeded version and inserting as new collection to qdrant database
"""

# [m for m in NVIDIAEmbeddings.get_available_models() if "embed" in m.id]


def ReadPdf(path_to_pdf: str) -> List[Document]:
    """
    This function returns list of langchain documents.
    Loading whole file as one document in order to split it with chunk overlap.
    """
    try:
        loader = PyPDFLoader(file_path=path_to_pdf, mode="single")
        print("Loading pdf file...")
        document = loader.load()
        return document

    except ValueError as e:
        print(
            "\033[91mValueError in ReadPdf: Provided path does not lead to a file: \033[0m",
            e,
        )
        return []


def EmbedDocument(file_path: str) -> tuple[list[list[float]], list[Document]]:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,
        chunk_overlap=50,
        length_function=len,
        is_separator_regex=True,
        separators=["\n\n","Art.[0-9]*","\n"],
    )

    doc = ReadPdf(file_path)
    doc_splitted = splitter.split_documents(documents=doc)
    doc_splitted_txt = [doc.page_content for doc in doc_splitted]

    print("Embedding...")
    embedder = NVIDIAEmbeddings(model=os.environ["EMBEDDER"])
    embeddings = embedder.embed_documents(doc_splitted_txt)
    # print("First vector: ", embeddings[0])
    return embeddings, doc_splitted


def InsertToVectorDb(
    embeddings: list[list[float]], doc_splitted: list[Document]
) -> bool:
    client = QdrantClient(url=os.getenv("QDRANT_ADDRESS"))

    if client.collection_exists(collection_name):
        print("Deleteing old collection...")
        client.delete_collection(collection_name=collection_name)

    print("Creating collection...")
    max_len = -1
    for i in doc_splitted:
        max_len=max(max_len, len(i.page_content))
        
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=max_len, distance=Distance.COSINE),
        timeout=30,
    )

    print("Starting adding new vectors")
    for index, chunk in enumerate(doc_splitted):
        if index % 100 == 0:
            print(f"Runnig {round(index / len(doc_splitted), 4) * 100}%")
        try:
            point = PointStruct(
                id=index, vector=embeddings[index], payload={"content": chunk}
            )
        except Exception as e:
            print(index, e)

        client.upsert(collection_name=collection_name, points=[point])
    client.close()


load_dotenv()
embeddings, doc_splitted = EmbedDocument("../data/prawod_wodne.pdf")
InsertToVectorDb(embeddings, doc_splitted)


Loading pdf file...
Embedding...


KeyboardInterrupt: 

In [130]:
for i in doc_splitted:
    print(len(i.page_content))

179
319
366
344
362
371
388
270
243
395
348
350
380
369
329
385
327
337
341
158
379
371
392
358
388
384
391
378
350
378
332
328
286
392
350
392
376
379
342
349
385
372
382
396
397
357
344
333
385
361
366
379
331
344
392
341
395
380
335
346
395
369
384
368
375
393
367
379
326
389
344
396
353
369
386
376
382
361
377
330
383
338
347
354
394
328
364
370
334
379
331
346
393
336
378
372
359
378
396
386
366
393
398
377
371
379
368
332
386
361
343
385
347
341
341
392
333
395
370
357
385
386
330
345
381
398
349
339
392
359
375
240
370
347
375
338
369
350
175
302
108
346
369
31
341
392
63
283
391
222
148
376
340
120
364
332
355
339
351
89
370
371
225
356
350
388
392
365
390
330
277
372
391
343
379
119
365
371
371
358
378
388
201
382
383
356
383
360
385
323
393
392
338
331
376
337
66
369
377
357
342
259
369
359
394
385
354
347
345
361
352
345
340
382
385
354
330
330
385
388
334
386
333
326
397
369
313
373
338
390
390
354
343
103
287
331
364
319
148
367
319
296
350
378
38
379
348
332
359
124
337
3

In [131]:
max_len = - 1
max_index = 0
for index, i in enumerate(doc_splitted):
    if max_len != max(max_len, len(i.page_content)):
        max_index = index
        max_len=max(max_len, len(i.page_content))
print(max_len)
print(max_index)

400
902


In [126]:
doc_splitted_2[13].page_content

'Art. 16. Ilekroć w ustawie jest mowa o: \n1) budowlach przeciwpowodziowych rozumie się przez to: \na) kanały ulgi, \nb) kierownice w ujściach rzek do morza, \nc) poldery przeciwpowodziowe, \nd) sztuczne zbiorniki przeciwpowodziowe, \ne) suche zbiorniki przeciwpowodziowe, \nf) wały przeciwpowodziowe, \ng) budowle regulacyjne, \nh) wrota przeciwpowodziowe i przeciwsztormowe, \ni) falochrony, \nj) budowle ochrony brzegów morskich, \nk) stopnie wodne \n– wraz z  obiektami związanymi z  nimi technicznie i  funkcjonalnie lub \nnieruchomościami przeznaczonymi na potrzeby ochrony przed powodzią; \n2) budowlach piętrzących – rozumie się przez to budowle umożliwiające stałe \nlub okresowe piętrzenie wód powierzchniowych ponad przyległy teren lub \nnaturalny poziom zwierciadła wód; \n3) celach środowiskowych dla wód morskich – rozumie się przez to: \na) pożądany stan podstawowych cech i  właściwości wód morskich, w  tym \ndna i  skały macierzystej znajdujących się na obszarze morza \nterytorialn